In [ ]:
import pymc as pm
import numpy as np
import pandas as pd 
import multiprocessing as mp
import platform
if platform.system() != "Windows": mp.set_start_method('spawn')

from tb_incubator.constants import set_project_base_path, ImplementCDR, BURN_IN
from tb_incubator.calibrate import get_bcm, get_all_priors
from tb_incubator.utils import get_next_run_number, get_next_run_number_for_config

from estival.wrappers import pymc as epm
from estival.sampling import tools as esamp
import arviz as az

from estival.utils.sample import SampleTypes
import nevergrad as ng
from estival.wrappers import nevergrad as eng
from estival.utils.parallel import map_parallel

pd.options.plotting.backend = "plotly"  
project_paths = set_project_base_path("../tb_incubator/")
calib_out = project_paths["OUT_PATH"]


In [ ]:
params = {
    "start_population_size": 1.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0
}

In [ ]:
burn_in = 10000
tune = 10000
draws = 25000

In [ ]:
def calibrate(
    out_path, params, draws, tune, 
    covid_effects = True, apply_diagnostic_capacity = True, xpert_improvement=True, apply_cdr=ImplementCDR.ON_NOTIFICATION
):
    bcm = get_bcm(
        params = params, 
        covid_effects = covid_effects, 
        apply_diagnostic_capacity = apply_diagnostic_capacity,
        xpert_improvement = xpert_improvement,
        apply_cdr = apply_cdr)
    run_number = get_next_run_number(out_path, draws, tune)
    file_suffix = f'{draws}d{tune}t_{run_number}'

    def optimize_ng_with_idx(item):
        idx, sample = item
        opt = eng.optimize_model(bcm, budget=500, opt_class=ng.optimizers.TwoPointsDE, suggested = sample, num_workers=8)
        rec= opt.minimize(500)
        return idx, rec.value[1]

    lhs_samples = bcm.sample.lhs(16, ci=0.67)
    lhs_lle = esamp.likelihood_extras_for_samples(lhs_samples, bcm)
    lhs_sorted = lhs_lle.sort_values("loglikelihood", ascending=False)
    opt_samples_idx = map_parallel(optimize_ng_with_idx, lhs_sorted.iterrows())
    best_opt_samps = bcm.sample.convert(opt_samples_idx)
    init_samps = best_opt_samps.convert(SampleTypes.LIST_OF_DICTS)[0:8]

    with pm.Model() as model:
        variables = epm.use_model(bcm)
        idata = pm.sample(step=[pm.DEMetropolisZ(variables)], draws=draws, tune=tune,cores=16,chains=8, initvals=init_samps, mp_ctx="spawn")
    idata.to_netcdf(str(out_path / f'calib_full_out_{file_suffix}.nc'))
    
    burnt_idata = idata.sel(draw=np.s_[burn_in:])
    idata_extract = az.extract(burnt_idata, num_samples=1000)
    bcm.sample.convert(idata_extract).to_hdf5(out_path / f'calib_extract_out_{file_suffix}.h5')

    spaghetti_res = esamp.model_results_for_samples(idata_extract, bcm) 
    spaghetti_res.results.to_hdf(path_or_buf = str(out_path / f"results_{file_suffix}.hdf"), key = "spaghetti")

    like_df = esamp.likelihood_extras_for_idata(idata, bcm)
    like_df.to_hdf(path_or_buf= str(out_path / f'results_{file_suffix}.hdf'), key = 'likelihood')

    with open(out_path / f'config_for{file_suffix}.txt', 'w') as file:
        # Header
        file.write(f"Calibration Configuration - {file_suffix}\n")
        file.write("=" * 50 + "\n\n")
        
        # Model settings
        file.write("MODEL SETTINGS:\n")
        file.write(f"  COVID effects: {covid_effects}\n")
        file.write(f"  Diagnostic capacity: {apply_diagnostic_capacity}\n")
        file.write(f"  CDR implementation: {apply_cdr}\n")
        file.write(f"  Xpert improvement: {xpert_improvement}\n")
        
        # Calibration settings
        file.write(f"\nCALIBRATION SETTINGS:\n")
        file.write(f"  Draws: {draws}\n")
        file.write(f"  Tune: {tune}\n")
        
        # Priors
        priors = get_all_priors(
            covid_effects=covid_effects,
            apply_diagnostic_capacity=apply_diagnostic_capacity,
            apply_cdr=apply_cdr, 
            xpert_improvement=xpert_improvement
        )
        
        for prior in priors:
            bounds = prior.bounds()
            file.write(f"  {prior.name}: {bounds[0]:.3f} to {bounds[1]:.3f}\n")


In [ ]:
def calibrate_with_configs(out_path, params, draws, tune, model_configs, covid_effects = True):
    for config_name, model_config in model_configs.items():
        bcm = get_bcm(
            params = params, 
            covid_effects = covid_effects, 
            **model_config)

        run_number = get_next_run_number_for_config(out_path, draws, tune, model_configs)
        file_suffix = f'{config_name}_{draws}d{tune}t_{run_number}nr'

        def optimize_ng_with_idx(item):
            idx, sample = item
            opt = eng.optimize_model(bcm, budget=500, opt_class=ng.optimizers.TwoPointsDE, suggested = sample, num_workers=8)
            rec= opt.minimize(500)
            return idx, rec.value[1]

        lhs_samples = bcm.sample.lhs(16, ci=0.67)
        lhs_lle = esamp.likelihood_extras_for_samples(lhs_samples, bcm)
        lhs_sorted = lhs_lle.sort_values("loglikelihood", ascending=False)
        opt_samples_idx = map_parallel(optimize_ng_with_idx, lhs_sorted.iterrows())
        best_opt_samps = bcm.sample.convert(opt_samples_idx)
        init_samps = best_opt_samps.convert(SampleTypes.LIST_OF_DICTS)[0:8]

        with pm.Model() as model:
            variables = epm.use_model(bcm)
            idata_raw = pm.sample(step=[pm.DEMetropolisZ(variables)], draws=draws, tune=tune,cores=16,chains=8, initvals=init_samps, mp_ctx="spawn")
        idata_raw.to_netcdf(str(out_path / f'calib_full_out_{file_suffix}.nc'))
        
        burnt_idata = idata_raw.sel(draw=np.s_[BURN_IN:])
        idata_extract = az.extract(burnt_idata, num_samples=1000)
        bcm.sample.convert(idata_extract).to_hdf5(out_path / f'calib_extract_out_{file_suffix}.h5')

        spaghetti_res = esamp.model_results_for_samples(idata_extract, bcm) 
        spaghetti_res.results.to_hdf(path_or_buf = str(out_path / f"results_{file_suffix}.hdf"), key = "spaghetti")

        like_df = esamp.likelihood_extras_for_idata(idata_raw, bcm)
        like_df.to_hdf(path_or_buf= str(out_path / f'results_{file_suffix}.hdf'), key = 'likelihood')
        
        # Extract config values from model_config
        apply_diagnostic_capacity = model_config['apply_diagnostic_capacity']
        apply_cdr = model_config['apply_cdr']
        xpert_improvement = model_config['xpert_improvement']

        with open(out_path / f'config_for{file_suffix}.txt', 'w') as file:
            # Header
            file.write(f"Calibration Configuration - {file_suffix}\n")
            file.write("=" * 50 + "\n\n")
            
            # Model settings
            file.write(f"  Config name: {config_name}\n")
            file.write("MODEL SETTINGS:\n")
            file.write(f"  COVID effects: {covid_effects}\n")
            file.write(f"  Diag. capacity: {apply_diagnostic_capacity}\n")
            file.write(f"  CDR implementation: {apply_cdr}\n")
            file.write(f"  Xpert improvement: {xpert_improvement}\n")
            
            # Calibration settings
            file.write(f"\nCALIBRATION SETTINGS:\n")
            file.write(f"  Draws: {draws}\n")
            file.write(f"  Tune: {tune}\n")
            
            # Priors
            priors = get_all_priors(
                covid_effects=covid_effects,
                apply_diagnostic_capacity=apply_diagnostic_capacity,
                apply_cdr=apply_cdr, 
                xpert_improvement=xpert_improvement
            )
            
            for prior in priors:
                bounds = prior.bounds()
                file.write(f"  {prior.name}: {bounds[0]:.3f} to {bounds[1]:.3f}\n")


## Configurations

In [ ]:
model_configs = {
    'cdr_notif': { 
        'apply_diagnostic_capacity': True,
        'xpert_improvement': True,
        'apply_cdr': ImplementCDR.ON_NOTIFICATION,
    },
    'cdr_detect': {
        'apply_diagnostic_capacity': True,
        'xpert_improvement': True,
        'apply_cdr': ImplementCDR.ON_DETECTION,
    },
    'no_cdr': {
        'apply_diagnostic_capacity': True,
        'xpert_improvement': True,
        'apply_cdr': ImplementCDR.NONE,
    },
    'cdr_notif_noxpert': { 
        'apply_diagnostic_capacity': True,
        'xpert_improvement': False,
        'apply_cdr': ImplementCDR.ON_NOTIFICATION,
    },
    'cdr_detect_noxpert': {
        'apply_diagnostic_capacity': True,
        'xpert_improvement': False,
        'apply_cdr': ImplementCDR.ON_DETECTION,
    },
    'no_cdr_noxpert': {
        'apply_diagnostic_capacity': True,
        'xpert_improvement': False,
        'apply_cdr': ImplementCDR.NONE,
    },
}

In [ ]:
#calibrate(calib_out, params, draws, tune, covid_effects = True, apply_diagnostic_capacity = True, xpert_improvement = True, apply_cdr=ImplementCDR.ON_NOTIFICATION)
calibrate_with_configs(calib_out, params, draws, tune, model_configs, covid_effects=True)